In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import yfinance as yf
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Not_normal_wd/'

Mounted at /content/drive


In [4]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [5]:
filename = 'RSV_data.csv'
HV_data = pd.read_csv(path + filename)

In [6]:
set_index(HV_data)

In [7]:
tm_data = HV_data

In [8]:
HV_data = HV_data.drop(columns=['RSV'])

In [9]:
HV_data['RSV'] = tm_data['RSV']

In [10]:
HV_data

,gdp_growth,volume,"('GARCH', 'normal', 0)","('GARCH', 'normal', 1)","('GARCH', 'gaussian', 0)","('GARCH', 'gaussian', 1)","('GARCH', 'ged', 0)","('GARCH', 'ged', 1)","('FIGARCH', 'normal', 0)","('FIGARCH', 'normal', 1)",...,usd_eur,usd_jpy,usd_gbp,usd_cny,usd_cad,usd_mxn,gt_data,log_returns,"('APARCH', 'studentst', 1)",RSV
Date,,,,,,,,,,,,,,,,,,,,,
2000-02-02,28984.929215,2.945453e+09,0.039601,0.038301,0.039601,0.038301,0.040044,0.039199,0.038388,0.038388,...,2.781970,307.252039,4.529620,23.415205,4.090860,26.750733,209.303607,-0.005604,0.037128,0.032740
2000-02-03,28984.929215,2.935907e+09,0.037979,0.038355,0.037979,0.038355,0.041000,0.040608,0.038633,0.038633,...,2.799436,311.491144,4.529266,23.412659,4.119569,26.561052,209.303607,-0.008693,0.038336,0.033467
2000-02-04,28984.929215,3.274647e+09,0.036791,0.037755,0.036791,0.037755,0.038214,0.040159,0.036990,0.036990,...,2.728548,306.262089,4.474254,23.415452,4.106629,26.433950,209.303607,0.000572,0.049778,0.032057
2000-02-07,28984.929215,3.413593e+09,0.046593,0.055072,0.046593,0.055072,0.047968,0.057512,0.046474,0.046474,...,2.733357,299.491542,4.460147,23.414391,4.140641,26.339020,209.303607,0.027307,0.065732,0.034853
2000-02-08,28984.929215,2.868803e+09,0.049131,0.041984,0.049131,0.041984,0.049560,0.042487,0.046711,0.046711,...,2.730104,301.761354,4.470789,23.415982,4.142338,26.135551,209.303607,0.010478,0.043373,0.035254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,0.000000,1.609587e+08,0.000120,-0.000665,0.000120,-0.000665,0.000456,-0.000424,0.000121,0.000121,...,-0.002546,0.509117,-0.003041,0.005162,0.001980,-0.105925,0.404061,0.008246,-0.004076,0.000006
2024-08-27,0.000000,9.767973e+07,-0.003889,-0.004798,-0.003889,-0.004798,-0.003539,-0.002953,-0.004368,-0.004368,...,0.002051,-0.551543,-0.001202,-0.006788,-0.001556,-0.173948,0.404061,0.009336,-0.005604,-0.000017
2024-08-28,0.000000,2.498703e+08,0.000280,-0.001148,0.000280,-0.001148,0.000449,-0.000383,0.000483,0.000483,...,0.000849,0.247487,0.000849,0.002263,0.002051,-0.219910,0.707107,0.010307,-0.001642,0.000048


# Split train, validation and test data

In [11]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

In [12]:
def sliding_window_transform(X, y, timesteps):
  X_windows = []
  y_windows = []
  for i in range(len(X) - timesteps + 1):
    X_window = X[i:i + timesteps]
    y_window = y[i + timesteps - 1]
    X_windows.append(X_window)
    y_windows.append(y_window)

  return np.array(X_windows), np.array(y_windows)

In [13]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

In [14]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## RSV

In [15]:
daily_train, daily_val, daily_test = split_data(HV_data, 'RSV')

In [16]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [17]:
HV_X_train, HV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
HV_X_val, HV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
HV_X_test, HV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [18]:
# HV_X_train = HV_X_train.reshape(HV_X_train.shape[0], win_sz, -1)
# HV_X_val = HV_X_val.reshape(HV_X_val.shape[0], win_sz, -1)
# HV_X_test = HV_X_test.reshape(HV_X_test.shape[0], win_sz, -1)

### LSTM

In [ ]:
lstm_garch_model = Sequential()
lstm_garch_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(LSTM(16))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(Dense(5))

lstm_garch_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
lstm_garch_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 25, 48)              │          14,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,221 (75.08 KB)

 Trainable params: 19,221 (75.08 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lstm_history = lstm_garch_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 0.0066 - val_loss: 1.1097e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - loss: 9.1052e-05 - val_loss: 5.1735e-06
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 7.1336e-05 - val_loss: 4.9625e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 5.1469e-05 - val_loss: 4.9727e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - loss: 4.2721e-05 - val_loss: 3.6466e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 4.7878e-05 - val_loss: 2.4101e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 4.1347e-05 - val_loss: 3.0953e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 3.2048e-05 - val_loss: 4.5980e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 3.6944e-05 - val_loss: 2.2697e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 3.5711e-05 - val_loss: 1.9591e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step

In [ ]:
# Predictions
HV_lstm_predictions = lstm_garch_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_lstm_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_lstm_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_lstm_predictions)
print(f'MAPE: {mape}%')

MSE: 2.3267506141488733e-06
RMSE: 0.001525369009174132
MAE: 0.0013677358577675667
MAPE: 7457.865567349863%


### GRU

In [19]:
gru_model = Sequential()
gru_model.add(GRU(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(GRU(16))
gru_model.add(Dropout(0.1))
gru_model.add(Dense(5))

gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
gru_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 25, 48)              │          11,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,629 (57.14 KB)

 Trainable params: 14,629 (57.14 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
gru_history = gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - loss: 0.0560 - val_loss: 6.1343e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - loss: 2.8607e-04 - val_loss: 2.4601e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - loss: 1.3209e-04 - val_loss: 1.1010e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - loss: 8.6310e-05 - val_loss: 8.4314e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 45ms/step - loss: 6.5336e-05 - val_loss: 5.6662e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - loss: 5.6119e-05 - val_loss: 5.6572e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - loss: 6.0238e-05 - val_loss: 3.6890e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - loss: 5.2036e-05 - val_loss: 3.2625e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - loss: 5.2588e-05 - val_loss: 2.5483e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - loss: 5.0406e-05 - val_loss: 2.6062e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/st

In [22]:
# Predictions
HV_gru_predictions = gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step


In [23]:
mse = mean_squared_error(HV_y_test, HV_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 2.5573864480569353e-06
RMSE: 0.0015991830564563068
MAE: 0.001373610420296297
MAPE: 7642.33579490894%


### LSTM_GRU

In [ ]:
lstm_gru_model = Sequential()
lstm_gru_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(GRU(16))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(Dense(5))

lstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

In [ ]:
lstm_gru_model.summary()

In [ ]:
lstm_gru_history = lstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - loss: 0.0246 - val_loss: 2.9721e-04
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 5.0866e-04 - val_loss: 8.3940e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 2.3584e-04 - val_loss: 7.2060e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 1.3601e-04 - val_loss: 1.3846e-05
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 1.0222e-04 - val_loss: 3.9017e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - loss: 6.9343e-05 - val_loss: 1.0278e-05
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 6.2726e-05 - val_loss: 2.7753e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 5.5831e-05 - val_loss: 2.0435e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 4.6880e-05 - val_loss: 1.7410e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - loss: 4.3278e-05 - val_loss: 3.9505e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step

In [ ]:
# Predictions
HV_lstm_gru_predictions = lstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_lstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 5.316968741530013e-06
RMSE: 0.002305855316694873
MAE: 0.0018807072415792062
MAPE: 9693.418212872597%


In [ ]:
# path_1 = '/content/drive/MyDrive/Volatility/Predictions/RSV/'

# df_pred = pd.DataFrame(HV_lstm_gru_predictions[:,0], columns=['lstm_gru_pred'])
# filename = 'LSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)

### BiLSTM

In [ ]:
bilstm_model = Sequential()
bilstm_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Bidirectional(LSTM(16)))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Dense(5))

bilstm_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
bilstm_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ (None, 25, 96)              │          29,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 25, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 32)                  │          14,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,581 (174.14 KB)

 Trainable params: 44,581 (174.14 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
bilstm_history = bilstm_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 15s 44ms/step - loss: 0.0247 - val_loss: 2.5887e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - loss: 1.8603e-04 - val_loss: 1.3641e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - loss: 1.0170e-04 - val_loss: 8.4130e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - loss: 7.7310e-05 - val_loss: 4.6311e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - loss: 5.8207e-05 - val_loss: 3.2686e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - loss: 4.7356e-05 - val_loss: 2.1457e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - loss: 4.4115e-05 - val_loss: 1.5099e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - loss: 4.8948e-05 - val_loss: 2.4387e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 53ms/step - loss: 4.3195e-05 - val_loss: 1.6678e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - loss: 4.0633e-05 - val_loss: 9.3924e-07
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 51m

In [ ]:
# Predictions
HV_bilstm_predictions = bilstm_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_bilstm_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_predictions)
print(f'MAPE: {mape}%')

MSE: 3.8155311358207005e-06
RMSE: 0.0019533384591055134
MAE: 0.0006127552149807471
MAPE: 2879.532733753115%


### BiLSTM_GRU

In [ ]:
bilstm_gru_model = Sequential()
bilstm_gru_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(GRU(16))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(Dense(5))

bilstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
bilstm_gru_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional)      │ (None, 25, 96)              │          29,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 25, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 16)                  │           5,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 35,509 (138.71 KB)

 Trainable params: 35,509 (138.71 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
bilstm_gru_history = bilstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - loss: 0.0224 - val_loss: 9.5714e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - loss: 4.0086e-04 - val_loss: 6.4990e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 1.6273e-04 - val_loss: 1.6086e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 45ms/step - loss: 9.3612e-05 - val_loss: 3.1896e-05
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - loss: 7.0743e-05 - val_loss: 1.1476e-05
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - loss: 4.9543e-05 - val_loss: 4.0822e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - loss: 3.8572e-05 - val_loss: 2.7051e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - loss: 3.8302e-05 - val_loss: 3.6396e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 47ms/step - loss: 3.9543e-05 - val_loss: 4.1301e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - loss: 3.0176e-05 - val_loss: 1.6422e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms

In [ ]:
# Predictions
HV_bilstm_gru_predictions = bilstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 2.1218975076803888e-07
RMSE: 0.0004606405874084902
MAE: 0.0003673203574768147
MAPE: 1860.7385647695971%


In [ ]:
# path_1 = '/content/drive/MyDrive/Volatility/New_data_2000_2024/New_predictions/RSV/'

# df_pred = pd.DataFrame(HV_lstm_predictions[:,0], columns=['lstm_pred'])
# filename = 'LSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_gru_predictions[:,0], columns=['gru_pred'])
# filename = 'GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_lstm_gru_predictions[:,0], columns=['lstm_gru_pred'])
# filename = 'LSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_predictions[:,0], columns=['bilstm_pred'])
# filename = 'BiLSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_gru_predictions[:,0], columns=['bilstm_gru_pred'])
# filename = 'BiLSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)